In [31]:
# from hyperopt.pyll.base import scope
# from hyperopt.pyll.stochastic import sample
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [32]:
# import random
# import itertools
N_HYPEROPT_PROBES = 10
EARLY_STOPPING = 80
HOLDOUT_SEED = 123456
HOLDOUT_SIZE = 0.10
HYPEROPT_ALGO = tpe.suggest  #  tpe.suggest OR hyperopt.rand.suggest
DATASET = 'clean' # 'raw' | 'clean' | 'extended'
SEED0 = random.randint(1,1000000000)
NB_CV_FOLDS = 5

In [33]:
obj_call_count = 0
cur_best_score = 0

In [34]:
space_XGB ={
    'max_depth'        : hp.choice("max_depth",        np.arange(4, 7,    dtype=int)), 
    'learning_rate'    : hp.loguniform('learning_rate', -6.9, -2.3),
    'n_estimators'     : hp.loguniform('n_estimators', 100, 1000),
    'objective'        : 'binary:logistic',
    'booster'         : 'gbtree',       
    'reg_alpha'       : hp.uniform('reg_alpha', 1e-6, 1e-4 ),
    'reg_lambda'      : hp.uniform('reg_lambda', 1e-6, 1e-4 ), 
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.6, 0.8),
    'min_child_weight ': hp.uniform('min_child_weight', 0.6, 0.8),   
    'random_state'    : hp.randint('random_state',2000000)
   }


In [37]:
def objective_XGB(space):
    from sklearn.model_selection import cross_val_score
    from xgboost import XGBClassifier
   
    global obj_call_count, cur_best_score, X_train, y_train, test, X_val, y_val
    X_train = X_train.astype(int)
    y_train = y_train.astype(int)
    X_val = X_val.astype(int)
    y_val = y_val.astype(int)
    
    obj_call_count += 1
    print('\nLightGBM objective call #{} cur_best_score={:7.5f}'.format(obj_call_count,cur_best_score) )

    sorted_params = sorted(space.items(), key=lambda z: z[0])
    print('Params:', str.join(' ', ['{}={}'.format(k, v) for k, v in sorted_params if not k.startswith('column:')]))


    params = sample(space)
        
    mdl = XGBClassifier(**params)
    
    cv_score = cross_val_score(mdl, X_train, y_train).mean()

    print( 'CV finished ; \n cv_score={:7.5f}'.format(cv_score ) )
    
    _model = mdl.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='auc',early_stopping_rounds =30, verbose=True)
    
    params.update({'n_estimators': _model.best_iteration})
    
    predictions = _model.predict_proba(X)[:,1](X_val)
    
    score = roc_auc_score(y_val, predictions)
    print('valid score={}'.format(score))
    
    
    do_submit = score > 0.64

    if score > cur_best_score:
        cur_best_score = score
        print('NEW BEST SCORE={}'.format(cur_best_score))
        do_submit = True

    if do_submit:
        submit_guid = uuid4()

        print('Compute submissions guid={}'.format(submit_guid))

        y_submission = gbm_model.predict(test, num_iteration=n_rounds)
        submission_filename = 'xgb_score={:13.11f}_submission_guid={}.csv'.format(score,submit_guid)
        pd.DataFrame(
        {'id':test_id, 'target':y_submission}
        ).to_csv(submission_filename, index=False)
       
    loss = 1 - score
    return {'loss': loss, 'status': STATUS_OK}

In [38]:
test = df.copy()
trials = Trials()
best = fmin(fn=objective_XGB,
                     space=space_XGB,
                     algo=HYPEROPT_ALGO,
                     max_evals=N_HYPEROPT_PROBES,
                     trials=trials,
                     verbose=1)

print('-'*50)
print('The best params for XGB:')
print( best )
print('\n\n')


LightGBM objective call #2 cur_best_score=0.00000
Params: booster=gbtree colsample_bytree=0.6664217171825527 learning_rate=0.07813084968909742 max_depth=6 min_child_weight =0.6908962863713928 n_estimators=3.0680669056728787e+87 objective=binary:logistic random_state=1449611 reg_alpha=5.985028117282578e-05 reg_lambda=7.948777531002902e-05


TypeError: 'float' object cannot be interpreted as an integer

### SVC

In [ ]:
svc_params = {}
svc_params['C'] = 0.1
svc_params['kernel'] = 'poly'
svc_params['gamma'] = 27

In [ ]:
svc_model = SVC(**svc_params)

In [ ]:
cross_val_model(X, y, svc_model)